In [31]:
import os
import json
from pathlib import Path
import urllib
from tqdm import tqdm
import paths

In [32]:
base_melart_path = paths.OUTPUT_DATASETS_PATH / 'melart_mimic'
base_melart_path.mkdir(exist_ok=True)
kb_images_path = base_melart_path / 'kb_image'
kb_images_path.mkdir(exist_ok=True)
mention_images_path = base_melart_path / 'mention_image'
mention_images_path.mkdir(exist_ok=True)

In [33]:
candidates_folder = paths.CANDIDATES_FOLDER_PATH
candidate_images_path = paths.MELART_IMAGES_PATH

In [34]:
#list all candidate images
candidate_images = [f for f in os.listdir(candidate_images_path) if os.path.isfile(os.path.join(candidate_images_path, f))]
candidate_images[:10]

['Tramroad bridge from entrance.jpg',
 'Buick Skylark Sport Wagon 1st gen.jpg',
 'Jose Cifuentes 2023.jpg',
 'Bernard van Orley - Self-portrait on Exterior Panel of Triptych of Virtue of Patience.jpg',
 'JimmyCarterPortrait2.jpg',
 'Fuggerorum et Fuggerarum imagines - 005r.jpg',
 'Titian – Cardinal Pietro Bembo – Google Art Project.jpg',
 'Shotter Boys - The City Hall, Amsterdam - 1953P48 - Birmingham Museums Trust.jpg',
 'PinkyLeeLadyofBurlesque.jpg',
 'Simone Martini 040.jpg']

In [35]:
kb_images_path

PosixPath('output_datasets/melart_mimic/kb_image')

In [36]:
commons_prefix="http://commons.wikimedia.org/wiki/Special:FilePath/"

In [37]:
# read all the candidate json files from /hpi/fs00/share/fg-naumann/datasets/wikidata/slices/el_candidates and put them in a dictionary
candidate_counter=0
candidate_objects=[]
candidate_index={}
qid2id={}
for file in tqdm(list(candidates_folder.iterdir())):
    if not file.name.endswith('.json'):
        continue
    with open(file, 'r') as f:
        candidate = json.load(f)
        qid = file.name.split('.')[0]
        #get all image urls
        img_list=[]
        images=candidate.get('images', [])
        if images:
            counter = 0
            img_list=[]
            for image in images:
                try:
                    ext=image.split('.')[-1]
                    image_name=urllib.parse.unquote(image.split('/')[-1])
                    if image_name in candidate_images:
                        #create a symlink with the qid and a counter
                        new_img_name=f'{qid}_{counter}.{ext}'
                        dst=kb_images_path / new_img_name
                        if not os.path.islink(dst):
                            os.symlink(str(candidate_images_path / image_name), str(dst))
                        img_list.append(new_img_name)
                        counter+=1
                except Exception as e:
                    print(e)
                    pass
        candidate_types_dict=candidate.get('types', dict())
        types_label_list=candidate_types_dict.values()
        # if candidate.get('statements', {}).get('P31'):
        #     for entity_type in candidate['statements']['P31']:
        #         try:
        #             types_qid_list.append(entity_type["value"]["content"])
        #         except:
        #             pass
        """
        build an object like this
        {
            "qid": "Q99989",
            "id": 109974,
            "type": "entity",
            "entity_name": "Slenaken",
            "attr": "",
            "instance": "village",
            "image_list": [
                "Q99989_0.jpg"
            ],
            "desc": "town in the Netherlands"
        },
        """
        obj={}
        obj['qid']=qid
        obj['id']=candidate_counter
        obj['type']='entity'
        obj['entity_name']=candidate['labels']['main'] if candidate['labels'].get('main') else ''
        if not obj['entity_name']:
            #try the first alternative label
            obj['entity_name']=candidate['labels']['alt'][0] if candidate['labels'].get('alt') and len(candidate['labels']['alt'])>0 else ''
        obj["image_list"]=img_list
        obj['desc']=candidate.get('description', "")
        entity_types=types_label_list
        obj["instance"]=", ".join(entity_types)
        obj['attr']=f"{obj['entity_name']}. {obj['desc']}. Types: {obj['instance']}" #not exacly attributes but there is no homogeneous set of attributes througout the candidates
        candidate_objects.append(obj)
        candidate_index[qid]=obj
        qid2id[qid]=candidate_counter
        candidate_counter+=1

# write the candidate objects to a json file
with open(base_melart_path / 'kb_entity_desc.json', 'w') as f:
    json.dump(candidate_objects, f)

# write qid2id to a json file
with open(base_melart_path / 'qid2id.json', 'w') as f:
    json.dump(qid2id, f)

100%|██████████| 53901/53901 [00:05<00:00, 9118.13it/s]


In [38]:
candidate_counter

53901

In [39]:
len(candidate_objects)

53901

Read the file with the paintings and the sentence mentions

In [40]:
paintings_file = paths.COMBINED_ANNOTATIONS_PATH
paintings=None
with open(paintings_file, 'r') as f:
    paintings = json.load(f)

In [41]:
painting_images_path = paths.MELART_IMAGES_PATH

#read painting images from /hpi/fs00/share/fg-naumann/seminar-ws22-tagging-captioning-art/artpedia-data/images/
paining_images_dict={}
painting_images_path = Path("/hpi/fs00/share/fg-naumann/datasets/wikidata/slices/images/artpedia/combined")
for paiting_img_path in tqdm(list(painting_images_path.iterdir())):
    # if not paiting_img_path.name.endswith('.png') and not paiting_img_path.name.endswith('.PNG'):
    #     print(f'not a png file: {paiting_img_path.name}')
    #     continue
    filename = paiting_img_path.name
    filename=urllib.parse.unquote(filename)
    name=Path(filename).stem
    paining_images_dict[name]=paiting_img_path

list(paining_images_dict.items())[:10]

"""
    convert from
    { "P135": "", "P136": "religious art", "P180": { "http://www.wikidata.org/entity/Q942467": "Christ Child", "http://www.wikidata.org/entity/Q345": "Virgin Mary" }, "P170": { "http://www.wikidata.org/entity/Q15790": "Cimabue" }, "image_url": [ "http://commons.wikimedia.org/wiki/Special:FilePath/Cimabue%20032.jpg" ], "visual_sentences": [ "It is also stylistically earlier to that work, being painted without pseudo-perspective, and having the angels around the Virgin simply placed one above the other, rather than being spatially arranged.", "The throne is similar to the Maest\u00e0 painted by Cimabue in the Basilica of San Francesco di Assisi (1288\u20131292)." ], "contextual_sentences": [ "The Maest\u00e0 is a painting by the Italian artist   Cimabue, painted around 1280 and housed in Mus\u00e9e du Louvre of Paris, France.", "The work is considered to be from around 1280, thus preceding the Santa Trinita Maest\u00e0.", "This work established a new canon for the Madonna with Child theme, which was subsequently used by other painters, such as Duccio di Buoninsegna in his Rucellai Maest\u00e0." ], "title": "Maest\u00e0 (Cimabue)", "img_url": "https://upload.wikimedia.org/wikipedia/commons/8/8c/Cimabue_-_Maest%C3%A0_du_Louvre.jpg", "split": "train", "year": 1280, "visual_el_matches": [ [], [] ], "contextual_el_matches": [ [], [], [ { "qid": "http://www.wikidata.org/entity/Q345", "text": "Madonna", "start": 42, "end": 49 } ] ] }    
    to 
    {"id": qid+cons, "sentence": "This work established a new canon for the Madonna with Child theme, which was subsequently used by other painters, such as Duccio di Buoninsegna in his Rucellai Maest\u00e0.", "imgPath": "20168.jpg", "mentions": "Madonna", "entities": "Virgin Mary", "answer": "Q345"}
    with any visual or contextual sentence that has a corresponding match
    """

keys_list=list(paining_images_dict.keys())
keys_list.sort()
keys_list

In [42]:
mention_objects=[]

paintings_mentions_index={}

for qid,paiting_obj in tqdm(paintings.items()):
    img_url=paiting_obj.get("img_url",None)
    new_img_file_path=None
    if img_url:
        img_file_name=img_url.split('/')[-1]
        img_path=Path(urllib.parse.unquote(img_file_name))
        img_path= painting_images_path / img_path
        if img_path.exists():
            #img_path=paining_images_dict[img_name]
            extention=img_path.suffix
            new_img_name=f'{qid}{extention}'
            new_img_file_path=mention_images_path / new_img_name
            #if symlink already exists, delete it
            if new_img_file_path.exists():
                new_img_file_path.unlink()
            os.symlink(str(img_path), str(new_img_file_path))
        else:
            print(f'no image in dict for {qid} ({img_file_name})')
            continue
    else:
        print(f'no image for {qid}')
        continue
    imgPath=new_img_file_path.name if new_img_file_path else ''
    counter=0
    for field in ["visual_el_matches","contextual_el_matches"]:
        for i,el_matches in enumerate(paiting_obj[field]):
            if len(el_matches)>0:
                sentence=paiting_obj["visual_sentences"][i] if field=="visual_el_matches" else paiting_obj["contextual_sentences"][i]
                for el_match in el_matches:
                    id=f'{qid}_{counter}'
                    counter+=1
                    obj={}
                    obj['id']=id
                    obj['sentence']=sentence
                    obj['imgPath']=imgPath
                    obj['mentions']=el_match['text']
                    match_qid=el_match['qid'].split('/')[-1]
                    candidate_obj=candidate_index[match_qid]
                    entity_name=candidate_obj['entity_name']
                    obj['entities']=entity_name
                    obj['answer']=match_qid
                    mention_objects.append(obj)
                    painting_mentions=paintings_mentions_index.get(qid,[])
                    painting_mentions.append(obj)
                    paintings_mentions_index[qid]=painting_mentions

100%|██████████| 1616/1616 [00:00<00:00, 10084.70it/s]


In [43]:
# split the paintings according to the split property in the painting objects
train_paintings = {}
dev_paintings = {}
test_paintings = {}

for qid, painting_obj in paintings.items():
    if painting_obj['split'] == 'train':
        train_paintings[qid] = painting_obj
    elif painting_obj['split'] == 'val':
        dev_paintings[qid] = painting_obj
    elif painting_obj['split'] == 'test':
        test_paintings[qid] = painting_obj

# report sizes
print(f'train: {len(train_paintings)}, dev: {len(dev_paintings)}, test: {len(test_paintings)}')


train: 1188, dev: 328, test: 100


In [44]:
train_file = base_melart_path / 'MELART_train.json'
dev_file = base_melart_path / 'MELART_dev.json'
test_file = base_melart_path / 'MELART_test.json'

In [45]:
# write 3 json files called MELART_train.json, MELART_dev.json, MELART_test.json inside the MELART folder, using the paintings_mentions_index
with open(train_file, 'w') as f:
    #find all mentions for the train paintings
    train_mentions=[]
    counter=0
    for qid,painting_obj in train_paintings.items():
        mentions=paintings_mentions_index.get(qid,[])
        if len(mentions)>0:
            train_mentions.extend(mentions)
            counter+=1
    print(f'found {counter} paintings with mentions for train')
    json.dump(train_mentions, f)
with open(dev_file, 'w') as f:
    #find all mentions for the dev paintings
    dev_mentions=[]
    counter=0
    for qid,painting_obj in dev_paintings.items():
        mentions=paintings_mentions_index.get(qid,[])
        if len(mentions)>0:
            dev_mentions.extend(mentions)
            counter+=1
    print(f'found {counter} paintings with mentions for dev')
    json.dump(dev_mentions, f)
with open(test_file, 'w') as f:
    #find all mentions for the test paintings
    test_mentions=[]
    counter=0
    for qid,painting_obj in test_paintings.items():
        mentions=paintings_mentions_index.get(qid,[])
        if len(mentions)>0:
            test_mentions.extend(mentions)
            counter+=1
    print(f'found {counter} paintings with mentions for test')
    json.dump(test_mentions, f)


found 1188 paintings with mentions for train
found 328 paintings with mentions for dev
found 100 paintings with mentions for test


In [46]:
#remove the imgPath from all the mentions in the train, test and dev files
for path in [train_file,dev_file,test_file]:
    with open(path, 'r') as f:
        mentions = json.load(f)
    for mention in mentions:
        mention["imgPath"]=""
    file_name=path.name
    new_name=file_name.replace('.json','_no_imgPath.json')
    new_path=path.parent / new_name
    with open(new_path, 'w') as f:
        json.dump(mentions, f)

In [47]:
#print sizes
print(f'train: {len(train_mentions)}, dev: {len(dev_mentions)}, test: {len(test_mentions)}')

train: 4632, dev: 1308, test: 645


In [48]:
#create kb_entity.json removing descriptions from kb_entity_desc.json
kb_entity_path=base_melart_path / 'kb_entity.json'
kb_entity_desc_path=base_melart_path / 'kb_entity_desc.json'


In [49]:
kb_entity=[]
with open(kb_entity_desc_path, 'r') as f:
    kb_entity_desc = json.load(f)
for obj in kb_entity_desc:
    obj.pop('desc', None)
    kb_entity.append(obj)
with open(kb_entity_path, 'w') as f:
    json.dump(kb_entity, f)

In [50]:
#generate the same kb_entity and kb_entity_desc files but with no images
kb_entity_no_img=json.load(open(kb_entity_path))
kb_entity_desc_no_img=json.load(open(kb_entity_desc_path))
for obj in kb_entity_no_img:
    obj['image_list']=[]
for obj in kb_entity_desc_no_img:
    obj['image_list']=[]
with open(base_melart_path / 'kb_entity_no_img.json', 'w') as f:
    json.dump(kb_entity_no_img, f)
with open(base_melart_path / 'kb_entity_desc_no_img.json', 'w') as f:
    json.dump(kb_entity_desc_no_img, f)
